In [3]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [11]:
df_movies = pd.read_csv('datasets/MovieLens/movies.csv')

print("Archivo cargado correctamente")

Archivo cargado correctamente


In [5]:
df_ratings = pd.read_csv('datasets/MovieLens/ratings.csv')

print("Archivo cargado correctamente")

Archivo cargado correctamente


In [13]:
df_movies.set_index('movieId', inplace=True)

df_movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [14]:
df_ratings.drop('timestamp', axis=1, inplace=True)

df_ratings.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [17]:
df_prom_ratings = df_ratings.groupby('movieId')['rating'].mean().reset_index()

df_prom_ratings.sort_values(by='rating', ascending=False, axis=0, inplace=True)

df_prom_ratings


,movieId,rating
29523,136782,5.0
49654,186119,5.0
29643,137032,5.0
49041,184643,5.0
29646,137038,5.0
...,...,...
5693,5805,0.5
55757,199922,0.5
53387,194608,0.5
58517,207153,0.5


In [19]:
df_movies_rating = df_movies.merge(df_prom_ratings, on='movieId')
df_movies_rating.sort_values(by='rating', ascending=False, axis=0, inplace=True)
df_movies_rating.head()

,movieId,title,genres,rating
29523,136782,The Girl is in Trouble (2015),Thriller,5.0
49654,186119,A Gift Horse (2015),Children,5.0
29643,137032,The Perfect Neighbor (2005),Drama|Thriller,5.0
49041,184643,Relentless (2018),Thriller,5.0
29646,137038,The Perfect Wife (2001),Drama|Thriller,5.0


In [21]:
num_votos = df_ratings['movieId'].value_counts().reset_index()

df_movies_rating_final = df_movies_rating.merge(num_votos, on='movieId')
df_movies_rating_final.sort_values(by='count', ascending=False, axis=0, inplace=True)

df_movies_rating_final.head()

,movieId,title,genres,rating,count
2566,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.048011,81491
1532,318,"Shawshank Redemption, The (1994)",Crime|Drama,4.413576,81482
2038,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.188912,79672
2209,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.151342,74127
2201,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.154099,72674


In [22]:
df_movies_rating_filtered = df_movies_rating_final.loc[df_movies_rating_final['count'] >= 6000]
df_movies_rating_filtered.sort_values(by='rating', ascending=False, axis=0, inplace=True)

df_movies_rating_filtered.drop(['genres'], axis=1, inplace=True)



df_movies_rating_filtered.head()

C:\Users\mvill\AppData\Local\Temp\ipykernel_16060\1283463326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_rating_filtered.sort_values(by='rating', ascending=False, axis=0, inplace=True)
C:\Users\mvill\AppData\Local\Temp\ipykernel_16060\1283463326.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_rating_filtered.drop(['genres'], axis=1, inplace=True)


,movieId,title,rating,count
1532,318,"Shawshank Redemption, The (1994)",4.413576,81482
1633,858,"Godfather, The (1972)",4.324336,52498
1648,50,"Usual Suspects, The (1995)",4.284353,55366
1652,1221,"Godfather: Part II, The (1974)",4.261759,34188
1655,2019,Seven Samurai (Shichinin no samurai) (1954),4.254769,13367


In [145]:
df_movies_meta = pd.read_csv('datasets/IMBD/movies_metadata.csv',
                             low_memory=False)

print("Archivo cargado correctamente")

Archivo cargado correctamente


In [146]:
df_movies_meta = df_movies_meta[['id', 'title', 'budget', 'revenue', 'release_date']]

# Eliminar fila con id 1997-08-20

df_movies_meta.drop(df_movies_meta[df_movies_meta['id'].astype(str).str.contains('-')].index, inplace=True)

df_movies_meta['id'] = df_movies_meta['id'].astype(int)

df_movies_meta.head()

,id,title,budget,revenue,release_date
0,862,Toy Story,30000000,373554033.0,1995-10-30
1,8844,Jumanji,65000000,262797249.0,1995-12-15
2,15602,Grumpier Old Men,0,0.0,1995-12-22
3,31357,Waiting to Exhale,16000000,81452156.0,1995-12-22
4,11862,Father of the Bride Part II,0,76578911.0,1995-02-10


In [127]:
df_links = pd.read_csv('datasets/IMBD/links.csv')

print("Archivo cargado correctamente")

Archivo cargado correctamente


In [128]:
df_links = df_links[['movieId', 'tmdbId']]

# Cambia el nombre del a segunda columna para que coincida con el df_movies_meta
df_links.rename(columns={'tmdbId': 'id'}, inplace=True)

df_links.dropna(inplace=True)

df_links['id'] = df_links['id'].astype('int')

df_links.head()


,movieId,id
0,1,862
1,2,8844
2,3,15602
3,4,31357
4,5,11862


In [147]:
# Concatena los df de links y movies_meta usando Concat

df_movies_meta_link = pd.merge(df_movies_meta, df_links, on='id')

df_movies_meta_link.drop(['id'], axis=1, inplace=True)

df_movies_meta_link.head()

,title,budget,revenue,release_date,movieId
0,Toy Story,30000000,373554033.0,1995-10-30,1
1,Jumanji,65000000,262797249.0,1995-12-15,2
2,Grumpier Old Men,0,0.0,1995-12-22,3
3,Waiting to Exhale,16000000,81452156.0,1995-12-22,4
4,Father of the Bride Part II,0,76578911.0,1995-02-10,5


In [152]:
# Une el df filtrado y el df de movies_meta usando merge

df_final = df_movies_rating_filtered.merge(df_movies_meta_link, on='movieId')

In [153]:
df_final = df_final[['movieId', 'title_y', 'rating', 'count', 'budget', 'revenue', 'release_date']]

df_final.rename(columns={'title_y': 'title'}, inplace=True)

In [154]:
df_final.head(15)

,movieId,title,rating,count,budget,revenue,release_date
0,318,The Shawshank Redemption,4.413576,81482,25000000,28341469.0,1994-09-23
1,858,The Godfather,4.324336,52498,6000000,245066411.0,1972-03-14
2,50,The Usual Suspects,4.284353,55366,6000000,23341568.0,1995-07-19
3,1221,The Godfather: Part II,4.261759,34188,13000000,47542841.0,1974-12-20
4,2019,Seven Samurai,4.254769,13367,2000000,271841.0,1954-04-26
5,527,Schindler's List,4.247579,60411,22000000,321365567.0,1993-11-29
6,1203,12 Angry Men,4.243014,16569,350000,1000000.0,1957-03-25
7,904,Rear Window,4.237948,20162,1000000,36764313.0,1954-08-01
8,2959,Fight Club,4.228311,58773,63000000,100853753.0,1999-10-15
9,1193,One Flew Over the Cuckoo's Nest,4.218662,36058,3000000,108981275.0,1975-11-18


In [ ]:
df_ratings